# Hands-On Pertemuan 6: Data Processing dengan Apache Spark

## Tujuan:
- Memahami dan mempraktikkan data processing menggunakan Apache Spark.
- Menggunakan Spark untuk operasi data yang efisien pada dataset besar.
- Menerapkan teknik canggih dalam Spark untuk mengatasi kasus penggunaan nyata.

### 1. Pengenalan Spark DataFrames
Spark DataFrame menyediakan struktur data yang optimal dengan operasi yang dioptimalkan untuk pemrosesan data besar, yang sangat mirip dengan DataFrame di Pandas atau di RDBMS.

- **Tugas 1**: Buat DataFrame sederhana di Spark dan eksplorasi beberapa fungsi dasar yang tersedia.

In [33]:
# Contoh membuat DataFrame sederhana dan operasi dasar
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql import functions as F 
spark = SparkSession.builder.appName('HandsOnPertemuan6').getOrCreate()

data = [('Zidhan', 'Software Engineer', 9999),
        ('Faizal', 'Dosen', 4600),
        ('Ridwan', 'Daboy Dancer', 4100),
        ('Restu', 'CEO', 7000),
        ('Rio', 'Software Engineer', 9999)]
columns = ['Nama', 'Pekerjaan', 'Gaji']

df = spark.createDataFrame(data, schema=columns)
df.show()

+------+-----------------+----+
|  Nama|        Pekerjaan|Gaji|
+------+-----------------+----+
|Zidhan|Software Engineer|9999|
|Faizal|            Dosen|4600|
|Ridwan|     Daboy Dancer|4100|
| Restu|              CEO|7000|
|   Rio|Software Engineer|9999|
+------+-----------------+----+



### 2. Transformasi Dasar dengan DataFrames
Pemrosesan data meliputi transformasi seperti filtering, selections, dan aggregations. Spark menyediakan cara efisien untuk melaksanakan operasi ini.

- **Tugas 2**: Gunakan operasi filter, select, groupBy untuk mengekstrak informasi dari data, serta lakukan agregasi data untuk mendapatkan insight tentang dataset menggunakan perintah seperti mean, max, sum.

In [34]:
# Contoh operasi transformasi DataFrame
df.select('Nama', 'Gaji').show()
df.filter(df['Gaji'] > 5000).show()
df.groupBy('Pekerjaan').avg('Gaji').show()
df.groupBy('Pekerjaan').agg(
    F.mean('Gaji').alias('AverageGaji'),
    F.max('Gaji').alias('MaxGaji'),
    F.sum('Gaji').alias('TotalGaji')
).show()

+------+----+
|  Nama|Gaji|
+------+----+
|Zidhan|9999|
|Faizal|4600|
|Ridwan|4100|
| Restu|7000|
|   Rio|9999|
+------+----+

+------+-----------------+----+
|  Nama|        Pekerjaan|Gaji|
+------+-----------------+----+
|Zidhan|Software Engineer|9999|
| Restu|              CEO|7000|
|   Rio|Software Engineer|9999|
+------+-----------------+----+

+-----------------+---------+
|        Pekerjaan|avg(Gaji)|
+-----------------+---------+
|            Dosen|   4600.0|
|Software Engineer|   9999.0|
|     Daboy Dancer|   4100.0|
|              CEO|   7000.0|
+-----------------+---------+

+-----------------+-----------+-------+---------+
|        Pekerjaan|AverageGaji|MaxGaji|TotalGaji|
+-----------------+-----------+-------+---------+
|            Dosen|     4600.0|   4600|     4600|
|Software Engineer|     9999.0|   9999|    19998|
|     Daboy Dancer|     4100.0|   4100|     4100|
|              CEO|     7000.0|   7000|     7000|
+-----------------+-----------+-------+---------+



### 3. Bekerja dengan Tipe Data Kompleks
Spark mendukung tipe data yang kompleks seperti maps, arrays, dan structs yang memungkinkan operasi yang lebih kompleks pada dataset yang kompleks.

- **Tugas 3**: Eksplorasi bagaimana mengolah tipe data kompleks dalam Spark DataFrames.

In [35]:
df_with_bonus = df.withColumn('BonusGaji', df['Gaji'] * 0.1)

df_with_compensation = df_with_bonus.withColumn('TotalCompensation', df_with_bonus['Gaji'] + df_with_bonus['BonusGaji'])

df_with_compensation.show()

+------+-----------------+----+-----------------+-----------------+
|  Nama|        Pekerjaan|Gaji|        BonusGaji|TotalCompensation|
+------+-----------------+----+-----------------+-----------------+
|Zidhan|Software Engineer|9999|999.9000000000001|          10998.9|
|Faizal|            Dosen|4600|            460.0|           5060.0|
|Ridwan|     Daboy Dancer|4100|            410.0|           4510.0|
| Restu|              CEO|7000|            700.0|           7700.0|
|   Rio|Software Engineer|9999|999.9000000000001|          10998.9|
+------+-----------------+----+-----------------+-----------------+



### 4. Operasi Data Lanjutan
Menggunakan Spark untuk operasi lanjutan seperti window functions, user-defined functions (UDFs), dan mengoptimalkan query.

- **Tugas 4**: Implementasikan window function untuk menghitung running totals atau rangkings.

In [36]:
# Contoh menggunakan window functions
from pyspark.sql.window import Window
from pyspark.sql import functions as F

windowSpec = Window.partitionBy('Pekerjaan').orderBy('Gaji')
df_with_rank = df.withColumn('Rank', F.rank().over(windowSpec))
df_with_running_total = df_with_rank.withColumn('RunningTotal', F.sum('Gaji').over(windowSpec.rowsBetween(Window.unboundedPreceding, Window.currentRow)))

# Menampilkan DataFrame akhir
df_with_running_total.show()

+------+-----------------+----+----+------------+
|  Nama|        Pekerjaan|Gaji|Rank|RunningTotal|
+------+-----------------+----+----+------------+
| Restu|              CEO|7000|   1|        7000|
|Ridwan|     Daboy Dancer|4100|   1|        4100|
|Faizal|            Dosen|4600|   1|        4600|
|Zidhan|Software Engineer|9999|   1|        9999|
|   Rio|Software Engineer|9999|   1|       19998|
+------+-----------------+----+----+------------+



### 5. Kesimpulan dan Eksplorasi Lebih Lanjut
Review apa yang telah dipelajari tentang pemrosesan data menggunakan Spark dan eksplorasi teknik lebih lanjut untuk mengoptimalkan pemrosesan data Anda.
- **Tugas 5**: Buat ringkasan dari semua operasi yang telah dilakukan dan bagaimana teknik ini dapat diterapkan pada proyek data Anda.


Dalam praktikum ini, saya telah menjelajahi berbagai operasi dasar dan lanjutan menggunakan PySpark DataFrame, termasuk pembuatan DataFrame dari data karyawan, manipulasi kolom untuk menghitung bonus dan total kompensasi, serta penerapan window functions untuk menghitung ranking dan running totals. Teknik-teknik ini sangat berharga dalam analisis data, karena memungkinkan pengguna untuk melakukan pengelompokan dan agregasi yang kompleks dengan efisien. Dalam proyek nanti, seperti analisis karyawan atau laporan keuangan, teknik ini dapat diimplementasikan untuk mengidentifikasi pola, menghasilkan wawasan yang mendalam, dan mendukung pengambilan keputusan strategis. Dengan kemampuan untuk menangani data besar secara efektif, PySpark menjadi alat yang penting dalam memfasilitasi analisis data yang mendetail dan berbasis bukti dalam berbagai konteks bisnis.